In [1]:
!nvidia-smi


Mon Sep  4 19:05:44 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   38C    P0    66W / 250W |     10MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import mal_scraper
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import mal_scraper
from tqdm import tqdm

In [3]:
import numpy as np
from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#import pydotplus as pydot 
#import graphviz
from keras.utils import plot_model
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [4]:
knownscores  = pd.read_csv('mal_scores_train_nonzero_v2.csv')
known_no_scores= pd.read_csv('mal_scores_train_zero.csv')
anime_avgs = np.load('anime_avgs.npy').item()
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict()
review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')
user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [5]:

n_users = 229756
n_animes = 12826

In [6]:
userid2idx=np.load("user.npy").item()
animeid2idx=np.load("anime.npy").item()
anime_rev_dict=np.load("anime_rev_number.npy").item()


In [427]:
n_highest_rated_shows= 6
k_nearest_shows = 5
t_highest_predicted_score=10
l_max_reviews = 50000

In [487]:
user_to_rec=input("Mal Username to check") # we ask the wat user we are lookingat

Mal Username to checkTheBaronVK


In [488]:
user_to_rec

'TheBaronVK'

In [489]:
user_scores = knownscores[knownscores['userid']==user_to_rec]

In [490]:
userratinglist2 = []
if user_scores.shape[0]==0: # This means the user doesn't exist in our dataset and we need to scrape him. 
    reviewdict = {'userid':user_to_rec,'name':user_to_rec,'index':user_to_rec}
    useranime = mal_scraper.get_user_anime_list(user_to_rec)
    if useranime is None:
        print('User Has Not reviewed enough anime') # TODO Add seed shows
        pass
    else:
         for animeiter in range(len(useranime)):
            reviewdict = {'name':user_to_rec,'index':user_to_rec}
            reviewdict['unkey']= str(useranime[animeiter]['id_ref'])+str(user_to_rec)
            statusval = useranime[animeiter]['consumption_status'].value
            #userdict[animeid] = usertest[animenum]['id_ref']
            reviewdict['animescore'] =useranime[animeiter]['score']
            #userdict[animestart] =useranime[animeiter]['start_date']
            #userdict[animefinished] =useranime[animeiter]['finished_date']
            reviewdict['animeid'] =  str(useranime[animeiter]['id_ref'])
            reviewdict['status']= statusval
            userratinglist2.append(reviewdict)
    user_data= pd.DataFrame(userratinglist2) # this is the user's 0 and nonzero scores
    user_data['score']=user_data['animescore'] #standardize the column names
    user_scores=user_data[user_data['score']!=0] # scores
    user_zeroes =user_data[user_data['score']==0] #zero scores
    user_zeroes['animeid']=user_zeroes['animeid'].astype(int)
    useravg= user_scores['score'].mean()
    print(user_scores.head())
    user_scores['score_usr_scaled']=user_scores['score']-useravg
    print(user_scores.head())
    user_scores['animeid']=user_scores['animeid'].astype(int)
    user_scores['animeavg']= user_scores.animeid.map(anime_avgs)
    print(user_scores.head())
    user_scores['score_anime_scaled']=user_scores['score']-user_scores['animeavg']
    print(user_scores.head())
else:
    user_zeroes=known_no_scores[known_no_scores['userid']==user_to_rec]


ERROR:root:Unable to parse the date text "06-28-15" from an anime list
ERROR:root:Unable to parse the date text "01-13-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "12-21-14" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "09-20-14" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "06-26-15" from an anime list
ERROR:root:Unable to parse the date text "12-28-14" from an anime list
ERROR:root:Unable to parse the date text "12-15-14" from an anime list
ERROR:

   animeid  animescore       index        name     status            unkey  \
1     5342           7  TheBaronVK  TheBaronVK  CONSUMING   5342TheBaronVK   
24    2605          10  TheBaronVK  TheBaronVK  CONSUMING   2605TheBaronVK   
35   22199           8  TheBaronVK  TheBaronVK  COMPLETED  22199TheBaronVK   
36      47           8  TheBaronVK  TheBaronVK  COMPLETED     47TheBaronVK   
42   11111          10  TheBaronVK  TheBaronVK  COMPLETED  11111TheBaronVK   

    score  
1       7  
24     10  
35      8  
36      8  
42     10  


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   animeid  animescore       index        name     status            unkey  \
1     5342           7  TheBaronVK  TheBaronVK  CONSUMING   5342TheBaronVK   
24    2605          10  TheBaronVK  TheBaronVK  CONSUMING   2605TheBaronVK   
35   22199           8  TheBaronVK  TheBaronVK  COMPLETED  22199TheBaronVK   
36      47           8  TheBaronVK  TheBaronVK  COMPLETED     47TheBaronVK   
42   11111          10  TheBaronVK  TheBaronVK  COMPLETED  11111TheBaronVK   

    score  score_usr_scaled  
1       7         -1.294643  
24     10          1.705357  
35      8         -0.294643  
36      8         -0.294643  
42     10          1.705357  
    animeid  animescore       index        name     status            unkey  \
1      5342           7  TheBaronVK  TheBaronVK  CONSUMING   5342TheBaronVK   
24     2605          10  TheBaronVK  TheBaronVK  CONSUMING   2605TheBaronVK   
35    22199           8  TheBaronVK  TheBaronVK  COMPLETED  22199TheBaronVK   
36       47           8  TheBaronVK

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [491]:
possiblerecs=[]
#user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
seedshows = set(user_scores.sort_values('score_usr_scaled',ascending=False)[:n_highest_rated_shows].animeid.values).union(set(user_scores.sort_values('score_anime_scaled',ascending=False)[:n_highest_rated_shows].animeid.values))

In [492]:
print("seed shows are:")
for rec in seedshows:
    print(anime_names[rec])

seed shows are:
Toradora!
Kore wa Zombie Desu ka? of the Dead
Another
Kore wa Zombie Desu ka?
Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari
Sayonara Zetsubou Sensei
Boogiepop wa Warawanai: Boogiepop Phantom
Bakemonogatari
Fullmetal Alchemist: Brotherhood
Mahou Shoujo Madoka★Magica
Suzumiya Haruhi no Yuuutsu (2009)


In [493]:
possiblerecs1=[]
for show in seedshows:
    topn =list(user_review_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
    print(show)
    possiblerecs1 = possiblerecs1+topn
    #print(type(show))

4224
10790
11111
8841
11979
2605
369
5081
5114
9756
4382


In [494]:
len(possiblerecs1)#so now we have one third of the target shows to check. 

55

In [495]:
#review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [496]:
review_nlp_numbers.head()

,1,32772,6,7,8,15,16,17,18,19,...,25157,25159,25161,25183,25241,25271,25283,25285,25303,25313
1,1.000000,0.772273,0.961916,0.958079,0.789438,0.937363,0.940282,0.920310,0.902537,0.970917,...,0.908150,0.934710,0.834914,0.969773,0.467217,0.365232,0.934109,0.425164,0.550415,0.646103
32772,0.772273,1.000000,0.768938,0.743720,0.684533,0.798572,0.755860,0.741147,0.745097,0.771846,...,0.767707,0.756733,0.652148,0.781312,0.440492,0.387374,0.800241,0.427720,0.424690,0.552193
6,0.961916,0.768938,1.000000,0.937337,0.792106,0.948547,0.933387,0.926542,0.901927,0.976784,...,0.910146,0.931709,0.839640,0.958035,0.476516,0.350941,0.927069,0.433980,0.522717,0.619298
7,0.958079,0.743720,0.937337,1.000000,0.767137,0.910258,0.932606,0.892298,0.883138,0.944207,...,0.896787,0.931441,0.827226,0.961775,0.439031,0.351537,0.927716,0.439281,0.527624,0.625555
8,0.789438,0.684533,0.792106,0.767137,1.000000,0.780716,0.774188,0.761989,0.726089,0.802190,...,0.762219,0.775014,0.668251,0.789861,0.378482,0.322515,0.785225,0.351967,0.465397,0.517582


In [497]:
possiblerecs2 = []
for show in seedshows:
    try:
        topk =list(review_nlp_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
        possiblerecs2 = possiblerecs2+topk
    except:
        continue
    #print(type(show))

In [498]:
#review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')

In [499]:
possiblerecs3 = []
for show in seedshows:
    try:
        topk =list(review_lf_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
        possiblerecs3 = possiblerecs3+topk
    except:
        continue
    #print(type(show))

In [500]:

user_known_anime = list(user_scores.animeid.values)+ list(user_zeroes.animeid.values)

In [501]:
possiblerecs1=[anime for anime in possiblerecs1 if anime not in user_known_anime ]
possiblerecs2=[anime for anime in possiblerecs2 if anime not in user_known_anime ]
possiblerecs3=[anime for anime in possiblerecs3 if anime not in user_known_anime ]
possiblerecs = possiblerecs1+possiblerecs2+possiblerecs3

In [502]:
possiblerecs= list(set(possiblerecs))

In [503]:
possiblerecs= list(set(possiblerecs))

In [504]:
len(possiblerecs)

83

In [505]:
print("Possible recs are:")
for rec in possiblerecs:
    print(anime_names[rec])
    

Possible recs are:
Busou Renkin
K
Zan Sayonara Zetsubou Sensei Bangaichi
Dakara Boku wa, H ga Dekinai.
Cross Ange: Tenshi to Ryuu no Rondo
Kantai Collection: KanColle
Goku Sayonara Zetsubou Sensei
Angel Beats!
Tonari no Kaibutsu-kun
Ore no Nounai Sentakushi ga, Gakuen Love Comedy wo Zenryoku de Jama Shiteiru
Ookami to Koushinryou
Kimi ga Nozomu Eien
Texhnolyze
Fairy Tail (2014)
Zoku Sayonara Zetsubou Sensei
Ore no Kanojo to Osananajimi ga Shuraba Sugiru
Good Morning Call
Ghost Hunt
Kekkai Sensen
Ookamikakushi
Gabriel DropOut Specials
Zero no Tsukaima
Kennosuke-sama
Fairy Tail
Yuuki Yuuna wa Yuusha de Aru
Escha & Logy no Atelier: Tasogare no Sora no Renkinjutsushi
Zero no Tsukaima: Futatsuki no Kishi
Another: The Other - Inga
Hunter x Hunter (2011)
Gokukoku no Brynhildr
Hidan no Aria
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
ef: A Tale of Melodies.
Sakurasou no Pet na Kanojo
Ao no Exorcist
Ore no Imouto ga Konnani Kawaii Wake ga Nai
Mousou Dairinin
Sora no Otoshimono
Charlotte
Mobile P

In [506]:
print("Possible recs1(User Scores) are:")
for rec in set(possiblerecs1):
    print(anime_names[rec])

Possible recs1(User Scores) are:
Zan Sayonara Zetsubou Sensei Bangaichi
Goku Sayonara Zetsubou Sensei
Angel Beats!
Texhnolyze
Zoku Sayonara Zetsubou Sensei
Ookamikakushi
Another: The Other - Inga
Hunter x Hunter (2011)
Mousou Dairinin
Sora no Otoshimono
Mobile Police Patlabor 2: The Movie
Nekomonogatari: Kuro
Kore wa Zombie Desu ka? of the Dead OVA
Clannad: After Story
Sora no Otoshimono: Forte
Lucky☆Star
Kore wa Zombie Desu ka? OVA
Elfen Lied
Sayonara Zetsubou Sensei Special
Zan Sayonara Zetsubou Sensei
Yonimo Osoroshii Grimm Douwa
Mirai Nikki (TV)
Suzumiya Haruhi-chan no Yuuutsu


In [507]:
print("Possible recs2(Tf-IDF) are:")
for rec in set(possiblerecs2):
    print(anime_names[rec])

Possible recs2(Tf-IDF) are:
K
Tonari no Kaibutsu-kun
Angel Beats!
Ore no Nounai Sentakushi ga, Gakuen Love Comedy wo Zenryoku de Jama Shiteiru
Ookami to Koushinryou
Clannad
Fairy Tail (2014)
Ghost Hunt
Zero no Tsukaima
Yuuki Yuuna wa Yuusha de Aru
Hidan no Aria
Gokukoku no Brynhildr
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Sakurasou no Pet na Kanojo
Mousou Dairinin
Charlotte
Baka to Test to Shoukanjuu
Baccano!
Nisekoi
Princess Tutu
Flip Flappers
Panty & Stocking with Garterbelt
Date A Live II
Date A Live
FLCL
Golden Time
Cardcaptor Sakura
Ao Haru Ride
Genei wo Kakeru Taiyou
Kaiba
Kekkai Sensen
Fruits Basket
Mahou Shoujo Lyrical Nanoha: The Movie 1st
Kanon (2006)


In [508]:
print("Possible recs3(Hidden Factors) are:")
for rec in set(possiblerecs3):
    print(anime_names[rec])

Possible recs3(Hidden Factors) are:
Busou Renkin
Cross Ange: Tenshi to Ryuu no Rondo
Dakara Boku wa, H ga Dekinai.
Kantai Collection: KanColle
Tonari no Kaibutsu-kun
Kimi ga Nozomu Eien
Ore no Kanojo to Osananajimi ga Shuraba Sugiru
Good Morning Call
Gabriel DropOut Specials
Zero no Tsukaima
Kennosuke-sama
Fairy Tail
Escha & Logy no Atelier: Tasogare no Sora no Renkinjutsushi
Zero no Tsukaima: Futatsuki no Kishi
ef: A Tale of Melodies.
Ao no Exorcist
Ore no Imouto ga Konnani Kawaii Wake ga Nai
Nikutai Ten'i
Danganronpa: Kibou no Gakuen to Zetsubou no Koukousei The Animation
Nisekoi
Tengen Toppa Gurren Lagann
Sukitte Ii na yo.
Angel Beats!: Another Epilogue
Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai
Naruto: Shippuuden Movie 5 - Blood Prison
Ansatsu Kyoushitsu (TV) 2nd Season: Kagaijugyou-hen
Elfen Lied
Fate/stay night
Hundred
Ao Haru Ride
Fate/kaleid liner Prisma☆Illya
Million Doll
Mirai Nikki (TV)
Aimai Elegy


In [509]:
ratings_score_test = pd.read_csv('mal_scores_test_nonzero_v2.csv')

In [510]:
ratings_score_test.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled,user_rev_count,anime_rev_count
0,0,0,13,13,889,9,SeinfeldHentai,SeinfeldHentai,COMPLETED,889SeinfeldHentai,SeinfeldHentai,9,9.111111,8.157417,0.842583,-0.111111,9,18397
1,1,1,16,16,28907,9,SeinfeldHentai,SeinfeldHentai,COMPLETED,28907SeinfeldHentai,SeinfeldHentai,9,9.111111,7.894818,1.105182,-0.111111,9,15497
2,2,2,233,233,21185,7,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,21185OPERATIONiDROID,OPERATIONiDROID,7,7.767857,7.921898,-0.921898,-0.767857,56,3329
3,3,3,243,243,18507,9,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,18507OPERATIONiDROID,OPERATIONiDROID,9,7.767857,7.631896,1.368104,1.232143,56,16786
4,4,4,248,248,15059,8,OPERATIONiDROID,OPERATIONiDROID,COMPLETED,15059OPERATIONiDROID,OPERATIONiDROID,8,7.767857,8.219309,-0.219309,0.232143,56,1564


In [511]:
recs=pd.DataFrame(possiblerecs, columns=['animeid'])

In [512]:
recs['userid']=user_to_rec

In [513]:
recs['animescore']=0
recs['score_anime_scaled']=0
recs['score_usr_scaled']=0


In [514]:
#recs.head()

In [515]:
#type(userid2idx)

In [516]:
#userid2idx

In [517]:
recs['user_id_emb'] = len(userid2idx)+2


In [518]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
user_scores['user_id_emb']= len(userid2idx)+1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [519]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
recs['anime_id_emb'] = recs.animeid.apply(lambda x: animeid2idx[x])



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [520]:
n_factors =40

In [521]:
def embedding_input_anime1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in1, u1 = embedding_input_user1('user_id_in', n_users+15, n_factors, 0)
u1.trainable=True
anime_in1, a1 = embedding_input_anime1('anime_id_in', n_animes, n_factors, 0)
a1.trainable=False

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [522]:
# nn1, It is given no modifications for status. It is predicting score
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dropout(0.55, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [523]:
nn1.load_weights('nn_score_weights_orig.h5',by_name=True)

In [524]:
!nvidia-smi

Mon Sep  4 19:25:19 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   43C    P2    59W / 250W |  10691MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [525]:
nn1.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score, epochs=9)

Epoch 1/9
112/112 [==============================] - 0s - loss: 1.4209     
Epoch 2/9
112/112 [==============================] - 0s - loss: 1.2587     
Epoch 3/9
112/112 [==============================] - 0s - loss: 1.2507     
Epoch 4/9
112/112 [==============================] - 0s - loss: 1.2629     
Epoch 5/9
112/112 [==============================] - 0s - loss: 1.1216     
Epoch 6/9
112/112 [==============================] - 0s - loss: 1.1908     
Epoch 7/9
112/112 [==============================] - 0s - loss: 1.1037     
Epoch 8/9
112/112 [==============================] - 0s - loss: 1.1141     
Epoch 9/9
112/112 [==============================] - 0s - loss: 1.0583     


In [526]:
recs['score']=nn1.predict([recs.user_id_emb, recs.anime_id_emb])

In [527]:
recs.head()

,animeid,userid,animescore,score_anime_scaled,score_usr_scaled,user_id_emb,anime_id_emb,score
0,1536,TheBaronVK,0,0,0,230964,1394,8.224501
1,14467,TheBaronVK,0,0,0,230964,7138,8.299364
2,7044,TheBaronVK,0,0,0,230964,4882,8.568973
3,12549,TheBaronVK,0,0,0,230964,6778,7.786059
4,25731,TheBaronVK,0,0,0,230964,9306,8.255509


In [528]:
recs.sort_values('score',ascending=False)

,animeid,userid,animescore,score_anime_scaled,score_usr_scaled,user_id_emb,anime_id_emb,score
28,11061,TheBaronVK,0,0,0,230964,6475,9.060782
50,4181,TheBaronVK,0,0,0,230964,3539,9.000514
49,2001,TheBaronVK,0,0,0,230964,1822,8.917624
31,11577,TheBaronVK,0,0,0,230964,6565,8.875997
42,2251,TheBaronVK,0,0,0,230964,2061,8.862906
74,3701,TheBaronVK,0,0,0,230964,3231,8.842714
10,2966,TheBaronVK,0,0,0,230964,2710,8.754132
33,13759,TheBaronVK,0,0,0,230964,7038,8.718456
77,4985,TheBaronVK,0,0,0,230964,3909,8.640657
75,2167,TheBaronVK,0,0,0,230964,1984,8.608368


In [529]:
def embedding_input_anime22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in22, u22 = embedding_input_user22('user_id_in', n_users+15, n_factors, 0)
u22.trainable=True
anime_in22, a22 = embedding_input_anime22('anime_id_in', n_animes, n_factors, 0)
a22.trainable=False
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u22, a22], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dropout(0.55, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn22 = Model([user_in22, anime_in22], x)
nn22.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.

In [530]:
nn22.load_weights('nn_score_usr_weights_orig.h5',by_name=True)

In [531]:
nn22.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score_usr_scaled, epochs=9)

Epoch 1/9
112/112 [==============================] - 0s - loss: 1.2844     
Epoch 2/9
112/112 [==============================] - 0s - loss: 1.1891     
Epoch 3/9
112/112 [==============================] - 0s - loss: 1.1665     
Epoch 4/9
112/112 [==============================] - 0s - loss: 1.1283     
Epoch 5/9
112/112 [==============================] - 0s - loss: 1.1496     
Epoch 6/9
112/112 [==============================] - 0s - loss: 1.1196     
Epoch 7/9
112/112 [==============================] - 0s - loss: 1.1112     
Epoch 8/9
112/112 [==============================] - 0s - loss: 1.0979     
Epoch 9/9
112/112 [==============================] - 0s - loss: 1.0740     


In [532]:
recs['score_usr_scaled']=nn22.predict([recs.user_id_emb, recs.anime_id_emb])

In [533]:
def embedding_input_anime23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in23, u23 = embedding_input_user23('user_id_in', n_users+15, n_factors, 0)
u23.trainable=True
anime_in23, a23 = embedding_input_anime23('anime_id_in', n_animes, n_factors, 0)
a23.trainable=False
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u23, a23], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dropout(0.55, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn23 = Model([user_in23, anime_in23], x)
nn23.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.

In [534]:
nn23.load_weights('nn_score_anime_weights_orig.h5',by_name=True)

In [535]:
nn23.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score_usr_scaled, epochs=9)

Epoch 1/9
112/112 [==============================] - 0s - loss: 1.2790     
Epoch 2/9
112/112 [==============================] - 0s - loss: 1.2679     
Epoch 3/9
112/112 [==============================] - 0s - loss: 1.2594     
Epoch 4/9
112/112 [==============================] - 0s - loss: 1.2273     
Epoch 5/9
112/112 [==============================] - 0s - loss: 1.1593     
Epoch 6/9
112/112 [==============================] - 0s - loss: 1.1388     
Epoch 7/9
112/112 [==============================] - 0s - loss: 1.1210     
Epoch 8/9
112/112 [==============================] - 0s - loss: 1.1617     
Epoch 9/9
112/112 [==============================] - 0s - loss: 1.1252     


In [536]:
recs['score_anime_scaled']=nn23.predict([recs.user_id_emb, recs.anime_id_emb])

In [537]:
recs['anime_rev_count']= recs.animeid.map(anime_rev_dict)

In [538]:
recs=recs[recs['anime_rev_count']<l_max_reviews]

In [539]:
recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)

,animeid,userid,animescore,score_anime_scaled,score_usr_scaled,user_id_emb,anime_id_emb,score,anime_rev_count
49,2001,TheBaronVK,0,0.403040,0.490750,230964,1822,8.917624,36141
28,11061,TheBaronVK,0,0.379204,0.485012,230964,6475,9.060782,30435
50,4181,TheBaronVK,0,0.245144,0.462498,230964,3539,9.000514,29410
42,2251,TheBaronVK,0,0.533969,0.450540,230964,2061,8.862906,17817
74,3701,TheBaronVK,0,0.693347,0.423020,230964,3231,8.842714,1711
31,11577,TheBaronVK,0,0.263668,0.413798,230964,6565,8.875997,12187
10,2966,TheBaronVK,0,0.345166,0.281141,230964,2710,8.754132,18096
77,4985,TheBaronVK,0,-0.007927,0.271657,230964,3909,8.640657,852
32,4789,TheBaronVK,0,0.180533,0.256001,230964,3819,8.551070,4129
40,15689,TheBaronVK,0,0.237293,0.247366,230964,7304,8.558910,11136


In [540]:
recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

,animeid,userid,animescore,score_anime_scaled,score_usr_scaled,user_id_emb,anime_id_emb,score,anime_rev_count
74,3701,TheBaronVK,0,0.693347,0.423020,230964,3231,8.842714,1711
36,323,TheBaronVK,0,0.624315,0.000386,230964,299,8.336361,5990
64,10469,TheBaronVK,0,0.606546,0.031378,230964,6226,8.459146,320
47,721,TheBaronVK,0,0.565827,0.211738,230964,663,8.555666,3210
2,7044,TheBaronVK,0,0.554440,0.225809,230964,4882,8.568973,802
62,227,TheBaronVK,0,0.550886,0.107015,230964,203,8.538313,18563
42,2251,TheBaronVK,0,0.533969,0.450540,230964,2061,8.862906,17817
68,6377,TheBaronVK,0,0.448002,0.183293,230964,4573,8.532577,2031
49,2001,TheBaronVK,0,0.403040,0.490750,230964,1822,8.917624,36141
14,3228,TheBaronVK,0,0.397747,0.226430,230964,2897,8.530801,3100


In [541]:
recs.sort_values('score',ascending=False).head(t_highest_predicted_score)

,animeid,userid,animescore,score_anime_scaled,score_usr_scaled,user_id_emb,anime_id_emb,score,anime_rev_count
28,11061,TheBaronVK,0,0.379204,0.485012,230964,6475,9.060782,30435
50,4181,TheBaronVK,0,0.245144,0.462498,230964,3539,9.000514,29410
49,2001,TheBaronVK,0,0.403040,0.490750,230964,1822,8.917624,36141
31,11577,TheBaronVK,0,0.263668,0.413798,230964,6565,8.875997,12187
42,2251,TheBaronVK,0,0.533969,0.450540,230964,2061,8.862906,17817
74,3701,TheBaronVK,0,0.693347,0.423020,230964,3231,8.842714,1711
10,2966,TheBaronVK,0,0.345166,0.281141,230964,2710,8.754132,18096
33,13759,TheBaronVK,0,0.012471,0.209150,230964,7038,8.718456,23994
77,4985,TheBaronVK,0,-0.007927,0.271657,230964,3909,8.640657,852
75,2167,TheBaronVK,0,0.024734,0.217767,230964,1984,8.608368,35023


In [542]:
score_user_preds =recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)
score_preds =recs.sort_values('score',ascending=False).head(t_highest_predicted_score)
score_anime_preds =recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

In [543]:
score_preds.animeid.values

array([11061,  4181,  2001, 11577,  2251,  3701,  2966, 13759,  4985,  2167])

In [544]:
print("Recommendations for score are:")
for rec in score_preds.animeid.values:
    print(anime_names[rec])

Recommendations for score are:
Hunter x Hunter (2011)
Clannad: After Story
Tengen Toppa Gurren Lagann
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Baccano!
Kaiba
Ookami to Koushinryou
Sakurasou no Pet na Kanojo
Mahou Shoujo Lyrical Nanoha: The Movie 1st
Clannad


In [545]:
print("Recommendations for  User_Scaled score are:")
for rec in score_user_preds.animeid.values:
    print(anime_names[rec])

Recommendations for  User_Scaled score are:
Tengen Toppa Gurren Lagann
Hunter x Hunter (2011)
Clannad: After Story
Baccano!
Kaiba
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Ookami to Koushinryou
Mahou Shoujo Lyrical Nanoha: The Movie 1st
ef: A Tale of Melodies.
Nekomonogatari: Kuro


In [546]:
print("Recommendations for  Anime_Scaled score are:")
for rec in score_anime_preds.animeid.values:
    print(anime_names[rec])

Recommendations for  Anime_Scaled score are:
Kaiba
Mousou Dairinin
Sayonara Zetsubou Sensei Special
Princess Tutu
Zan Sayonara Zetsubou Sensei Bangaichi
FLCL
Baccano!
Zan Sayonara Zetsubou Sensei
Tengen Toppa Gurren Lagann
Zoku Sayonara Zetsubou Sensei
